<a href="https://colab.research.google.com/github/CodeeSam/Github_Test/blob/main/Metagonome_pilot_study.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Installing necessary packages
!pip install biopython
!pip install transformers
!pip install tensorflow==2.11.0

# Importing required libraries
import os
import numpy as np
import pandas as pd
from Bio import SeqIO
from Bio.Seq import Seq
import tensorflow as tf
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification

print("Environment setup complete!")
print("TensorFlow version:", tf.__version__)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 77.4 MB/s eta 0:00:00
ERROR: Could not find a version that satisfies the requirement tensorflow==2.11.0 (from versions: 2.12.0rc0, 2.12.0rc1, 2.12.0, 2.12.1, 2.13.0rc0, 2.13.0rc1, 2.13.0rc2, 2.13.0, 2.13.1, 2.14.0rc0, 2.14.0rc1, 2.14.0, 2.14.1, 2.15.0rc0, 2.15.0rc1, 2.15.0, 2.15.0.post1, 2.15.1, 2.16.0rc0, 2.16.1, 2.16.2, 2.17.0rc0, 2.17.0rc1, 2.17.0, 2.17.1, 2.18.0rc0, 2.18.0rc1, 2.18.0rc2, 2.18.0, 2.18.1, 2.19.0rc0, 2.19.0)
ERROR: No matching distribution found for tensorflow==2.11.0
Environment setup complete!
TensorFlow version: 2.18.0


In [ ]:
# Updating package lists and installing EMBOSS (which provides getorf)
!apt-get update
!apt-get install -y emboss

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:5 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:10 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [2,737 kB]
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,240 kB]
Get:13 http://security.ubuntu.com/ubuntu jammy-securit

In [ ]:
os.getcwd()

'/content/sample_data'

In [ ]:
!ls -l

total 55548
-rwxr-xr-x 1 root root     1697 Jan  1  2000 anscombe.json
-rw-r--r-- 1 root root   301141 Mar 24 13:34 california_housing_test.csv
-rw-r--r-- 1 root root  1706430 Mar 24 13:34 california_housing_train.csv
-rw-r--r-- 1 root root 18289443 Mar 24 13:34 mnist_test.csv
-rw-r--r-- 1 root root 36523880 Mar 24 13:34 mnist_train_small.csv
-rw-r--r-- 1 root root      958 Mar 31 16:07 pilot_AMPs_test.fa.txt
-rw-r--r-- 1 root root     2814 Mar 31 16:07 pilot_Non-AMPs_test.fa.txt
-rw-r--r-- 1 root root    36321 Mar 31 16:07 pilot_pilot_study_seq.fasta.txt
-rwxr-xr-x 1 root root      962 Jan  1  2000 README.md


In [ ]:
# Running getorf on the test metagenome FASTA file to extract ORFs between 15 and 150 bp
!getorf -sequence pilot_pilot_study_seq.fasta -find 0 -table 11 -minsize 15 -maxsize 150 -outseq pilot_pilot_study_seq.orfs.fa

Find and extract open reading frames (ORFs)


In [ ]:
def load_fasta(filepath):
    """Load a FASTA file into a dictionary with record IDs as keys and sequences as values."""
    return {record.id: str(record.seq) for record in SeqIO.parse(filepath, "fasta")}

# Loading candidate ORFs and known AMPs
candidate_orfs = load_fasta("/content/sample_data/pilot_pilot_study_seq.orfs.fa")
known_amp = load_fasta("/content/sample_data/pilot_AMPs_test.fa")

# Filtering out candidates that exactly match any known AMP
filtered_candidates = {k: v for k, v in candidate_orfs.items() if v not in known_amp.values()}

print("Number of candidate ORFs before filtering:", len(candidate_orfs))
print("Number of candidate ORFs after filtering known AMPs:", len(filtered_candidates))


Number of candidate ORFs before filtering: 1207
Number of candidate ORFs after filtering known AMPs: 1207


In [ ]:
# Defining a function to read a FASTA file and tokenize sequences
def tokenize_sequences(fasta_file, max_length=512):
    #Initializing the tokenizer from a pre-trained model- ProtBert:
    # Make sure to replace 'Rostlab/prot_bert' with the desired pre-trained model name if necessary.
    tokenizer = AutoTokenizer.from_pretrained("Rostlab/prot_bert_bfd", do_lower_case=False)
    sequences = []
    ids = []
    for record in SeqIO.parse(fasta_file, "fasta"):
        # For protein models, space-separate each amino acid (required by many protein tokenizers)
        sequences.append(" ".join(list(str(record.seq))))
        ids.append(record.id)
    tokens = tokenizer(sequences, return_tensors="tf", padding='max_length', truncation=True, max_length=max_length)
    return tokens, ids

# Tokenize the known AMP (positive) and non-AMP (negative) sequences
tokens_pos, ids_pos = tokenize_sequences("/content/sample_data/pilot_AMPs_test.fa", max_length=64)
tokens_neg, ids_neg = tokenize_sequences("/content/sample_data/pilot_Non-AMPs_test.fa", max_length=64)

print("Tokenization complete for AMP and non-AMP sequences.")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/86.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/361 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/81.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Tokenization complete for AMP and non-AMP sequences.


In [ ]:
seq_object = SeqIO.parse('pilot_pilot_study_seq.fasta', 'fasta')
sequencess = []

for seq in seq_object:
    sequencess.append(seq)

sequencess
len(sequencess)

60

In [ ]:
# Load the pre-trained model and tokenizer from Hugging Face
#tokenizer = AutoTokenizer.from_pretrained("Rostlab/prot_bert_bfd", do_lower_case=False)
model = TFAutoModelForSequenceClassification.from_pretrained("Rostlab/prot_bert_bfd", num_labels=2)

# Prepare TensorFlow dataset for training
def prepare_dataset(tokens_pos, tokens_neg, batch_size=9):
    labels = tf.concat([
        tf.ones((tokens_pos['input_ids'].shape[0],), dtype=tf.int32),
        tf.zeros((tokens_neg['input_ids'].shape[0],), dtype=tf.int32)
    ], axis=0)
    input_ids = tf.concat([tokens_pos['input_ids'], tokens_neg['input_ids']], axis=0)
    attention_mask = tf.concat([tokens_pos['attention_mask'], tokens_neg['attention_mask']], axis=0)
    dataset = tf.data.Dataset.from_tensor_slices(({'input_ids': input_ids, 'attention_mask': attention_mask}, labels))
    return dataset.shuffle(10).batch(batch_size)

train_dataset = prepare_dataset(tokens_pos, tokens_neg, batch_size=9)

# Compile and fine-tune the model
#optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer='adam', loss=loss, metrics=['accuracy'])


model.summary()

# Fine-tune for a small number of epochs (for the pilot)
history = model.fit(train_dataset, epochs=5)


All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at Rostlab/prot_bert_bfd and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model: "tf_bert_for_sequence_classification_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  419931136 
                                                                 
 dropout_183 (Dropout)       multiple                  0 (unused)
                                                                 
 classifier (Dense)          multiple                  2050      
                                                                 
Total params: 419933186 (1.56 GB)
Trainable params: 419933186 (1.56 GB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/5
13/13 [==============================] - 458s 29s/step - loss: 0.9600 - accuracy: 0.8440
Epoch 2/5
13/13 [==============================] - 366s 28s/step - loss: 1.1902 - accuracy: 0.6789
Epoch 3/5
13/13 [==============================] - 36

In [20]:
# Tokenize candidate ORFs from filtered_candidates
def tokenize_candidate_sequences(candidate_dict, max_length=64):
    sequences = []
    ids = []
    for seq_id, seq in candidate_dict.items():
        sequences.append(" ".join(list(seq)))
        ids.append(seq_id)
    tokenizer = AutoTokenizer.from_pretrained("Rostlab/prot_bert_bfd", do_lower_case=False)
    tokens = tokenizer(sequences, return_tensors="tf", padding='max_length', truncation=True, max_length=max_length)
    return tokens, ids

tokens_candidates, candidate_ids = tokenize_candidate_sequences(filtered_candidates, max_length=64)

# Predict using the fine-tuned model
predictions = model.predict(tokens_candidates)
# Get probabilities from logits using softmax
probabilities = tf.nn.softmax(predictions.logits, axis=-1).numpy()[:, 1]  # probability for AMP class

# Create a DataFrame for results
results = pd.DataFrame({
    "Candidate_ID": candidate_ids,
    "Sequence": list(filtered_candidates.values()),
    "Prediction_Probability": probabilities
})

# Set threshold (0.5) for classification
results["Predicted_Label"] = (results["Prediction_Probability"] >= 0.5).astype(int)
print("Candidate AMP Predictions:")
print(results)

38/38 [==============================] - 1132s 30s/step
Candidate AMP Predictions:
              Candidate_ID                                      Sequence  \
0      JBEVUU010000001.1_1                               RALMVKVRLPTSWVC   
1      JBEVUU010000001.1_2                              KYGYRPHGCARQRGWD   
2      JBEVUU010000001.1_3                              NFDWWQYRCSSARSAL   
3      JBEVUU010000001.1_4                                SLCSGSGYLNFVCI   
4      JBEVUU010000001.1_5  ATRLGLKLRLVAISVFQRSICSLKILIFMFWFGIFKFCLYINLL   
...                    ...                                           ...   
1202  JBEVUU010000060.1_32                                     ICSVHCFMP   
1203  JBEVUU010000060.1_33                                    FVYTPHLECC   
1204  JBEVUU010000060.1_34                                   VIMERSFAINF   
1205  JBEVUU010000060.1_35                  WNGVSQSTFSRKMQTVWHYTKFYSQLSP   
1206  JBEVUU010000060.1_36                           QKDANSLALHKVLQSVITQ   

    

In [25]:
results.to_csv("/content/sample_data/candidate_amp_predictions.csv", index=False)
print("Candidate AMP predictions saved to 'Sample_data/candidate_amp_predictions.csv'.")

Candidate AMP predictions saved to 'Sample_data/candidate_amp_predictions.csv'.
